# 1-band map to RGB map converter

ToDo:

* label 3?
* hard-coded CSR
* 255 -> 0 nodata value


In [ ]:
#%matplotlib inline
import numpy as np
from pathlib import Path
import os
import rasterio as rio
import rasterio.plot as rioplot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

from skimage import data, img_as_float
from skimage import exposure
from PIL import Image

dictBands = {'b': (0,2), 'g': (1,1), 'r': (2,0)}

In [ ]:
plt.rcParams["figure.figsize"] = (20,12)

In [ ]:
DATADIR = "../../data/train"
RAW_MAPS = DATADIR + "/map"     #'../data/raw/images'
RGB_MAPS = DATADIR + "/map_rgb"      #'../data/raw/images'

In [ ]:
def street_mask(arr, value):
    arr = np.array(arr)
    ax = np.ma.masked_where(arr != value, arr)
    ax.fill_value = 0
    return (ax/ax*255).filled().astype(np.uint8)

def mask(arr, value):
    arr = np.array(arr)
    ax = np.ma.masked_where(arr == value, arr, copy = False)
    ax.fill_value = 0
    return (ax/ax*255).filled().astype(np.uint8)

In [ ]:
def map2rgbMap(inds):
    
    ima = inds.read()
    
    vals, counts = np.unique(ima, return_counts=True)
    
    dic = (dict(zip(vals, counts)))
    if 127 not in dic.keys():
        dic[127] = 0
    if 255 not in dic.keys():
        dic[255] = 0
    print(dic)
    
    sz  = ima.shape

    rgbMap = np.zeros((sz[1], sz[2], 4), 'uint8') 
    msk = mask(ima, 0)   # nodata value
    for index, i in enumerate(np.sort(vals[vals>0])[::-1]):        
        rgbMap[..., index] = street_mask(ima, i)
    rgbMap[..., 3] = msk
    return rgbMap, dic

In [ ]:
def write_visible(src, meta):    
    outpath = RGB_MAPS + '/' + file.name
    with rio.open(outpath, 'w', **meta) as outds:
        sz = np.shape(src)
        output  = np.zeros((4, sz[0], sz[1]), 'uint8')  
        for i in range(4):
            output[i] = src[..., i]      
        outds.write(output.astype(np.uint8))

In [ ]:
def make_visible(file):
    with rio.open(file) as inds:
        meta = inds.meta.copy() 
        meta['dtype']  = 'uint8'
        meta['count']  = 4
        meta['nodata'] = 0
        
        
        rgbMap, dic = map2rgbMap(inds)
        plt.imshow(rgbMap)
        plt.show()   
        
        write_visible(rgbMap, meta)
        
        return dic
        

In [ ]:
n = 1
stats = []
for file in Path(RAW_MAPS).iterdir():
    if file.name.endswith(('.tif', '.tiff')):
        dic = make_visible(file)
        #stats.append([file.name, dic[0], dic[127], dic[255]])

In [1]:
import pandas as pd

/home/lisa/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/lisa/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
df = pd.DataFrame.from_dict(stats)
df.columns = ['name', 'numPixel_noRoad', 'numPixel_pavedRoad', 'numPixel_unpavedRoad']

In [ ]:
#df.to_csv('numOfPixPerClassPerTile.csv')

In [2]:
#df = pd.read_csv('numOfPixPerClassPerTile.csv')

In [3]:
col_list= list(df)
col_list.remove('name')
col_list

In [7]:
df['sumPixels'] = df[col_list].sum(axis=1)

empty tiles:  0.42988364972443355
non-empty tiles:  0.5701163502755664

tiles with only paved roads:  0.016227801592161667
tiles with paved roads:  0.2676056338028169

tiles with only unpaved roads:  0.3025107164727495
tiles with unpaved roads:  0.5538885486834048


In [ ]:
df['relative_noRoad']    = df['numPixel_noRoad']/df['sumPixels']
df['relative_pavedRoad'] = df['numPixel_pavedRoad']/df['sumPixels']
df['relative_unpavedRoad'] = df['numPixel_unpavedRoad']/df['sumPixels']